In [1]:
import os
from dotenv import load_dotenv 
from datetime import datetime
import numpy as np
import pandas as pd
from fredapi import Fred

In [2]:
load_dotenv()
fred_key = os.getenv('FRED_API_KEY')
fred = Fred(api_key=fred_key)

In [3]:
country_id_map = {
    'Australia': 'IRLTLT01AUM156N',
    'Belgium': 'IRLTLT01BEM156N',
    'Canada': 'IRLTLT01CAM156N',
    'Denmark': 'IRLTLT01DKM156N',
    'France': 'IRLTLT01FRM156N',
    'Germany': 'IRLTLT01DEM156N',
    'Greece': 'IRLTLT01GRM156N',
    'India': 'INDIRLTLT01STM',
    'Ireland': 'IRLTLT01IEM156N',
    'Italy': 'IRLTLT01ITM156N',
    'Japan': 'IRLTLT01JPM156N',
    'Korea, Rep.': 'IRLTLT01KRM156N',
    'Netherlands': 'IRLTLT01NLM156N',
    'New Zealand': 'IRLTLT01NZM156N',
    'Norway': 'IRLTLT01NOM156N',
    'Poland': 'IRLTLT01PLM156N',
    'Portugal': 'IRLTLT01PLM156N',
    'South Africa': 'IRLTLT01ZAM156N',
    'Spain': 'IRLTLT01ESM156N',
    'Sweden': 'IRLTLT01SEM156N',
    'United Kingdom': 'IRLTLT01GBM156N',
    'United States': 'IRLTLT01USM156N'

}

In [4]:
fred.get_series('IRLTLT01AUM156N', observation_start = '11-01-2024')

2024-11-01    4.544
2024-12-01    4.313
2025-01-01    4.481
2025-02-01    4.423
2025-03-01    4.421
2025-04-01    4.267
2025-05-01    4.350
2025-06-01    4.220
dtype: float64

In [5]:
%cd ..
%cd data

credit_df = pd.read_csv('credit_ratings.csv')
credit_df.head()

/Users/jamiewong/Documents/esg-financial-assistant
/Users/jamiewong/Documents/esg-financial-assistant/data


,Country Name,Credit Rating
0,Australia,100
1,Belgium,87
2,Canada,100
3,Denmark,100
4,France,88


In [6]:
esg_df = pd.read_csv('country_esg_scores.csv')
esg_df.head()

,Country Name,E Score,S Score,G Score,ESG Score
0,Sweden,0.851642,0.845986,0.772606,2.470234
1,Norway,0.828819,0.799264,0.817535,2.445618
2,Denmark,0.750027,0.803905,0.846567,2.400499
3,New Zealand,0.594863,0.854180,0.848597,2.297640
4,Germany,0.767347,0.772925,0.736296,2.276569


In [7]:
merged_df = pd.merge(credit_df, esg_df, on = 'Country Name', how = 'left')
merged_df.head()

,Country Name,Credit Rating,E Score,S Score,G Score,ESG Score
0,Australia,100,0.547563,0.866146,0.821755,2.235464
1,Belgium,87,0.662349,0.773847,0.647842,2.084038
2,Canada,100,0.484865,0.779746,0.752887,2.017498
3,Denmark,100,0.750027,0.803905,0.846567,2.400499
4,France,88,0.797240,0.764189,0.625450,2.186880


In [8]:
def fix_date(mo, yr):
    arr = []
    arr.append(12 + mo)
    arr.append(yr - 1)
    return arr

In [9]:
latest_yr = datetime.now().year
latest_mo = datetime.now().month

In [10]:
yields_df = pd.DataFrame({'index_col': ['6mo_perf']})
for country in country_id_map:
    latest_data = fred.get_series(country_id_map[country], observation_start=f'{latest_mo}-01-{latest_yr}')
    while latest_data.empty:
        latest_mo -= 1
        if latest_mo < 1:
            latest_fixed = fix_date(latest_mo, latest_yr)
            latest_yr = latest_fixed[1]
            latest_mo = past_fixed[0]
        latest_data = fred.get_series(country_id_map[country], observation_start=f'{latest_mo}-01-{latest_yr}')

    past_yr = latest_yr
    past_mo = latest_mo - 6
    if past_mo < 1:
        past_fixed = fix_date(past_mo, past_yr)
        past_yr = past_fixed[1]
        past_mo = past_fixed[0]
    past_data = fred.get_series(country_id_map[country], observation_start=f'{past_mo}-01-{past_yr}', observation_end = f'{past_mo}-01-{past_yr}')
    yields_df[country] = (latest_data.iloc[0]-past_data.iloc[0])/past_data.iloc[0]

In [11]:
yields_df

,index_col,Australia,Belgium,Canada,Denmark,France,Germany,Greece,India,Ireland,...,Netherlands,New Zealand,Norway,Poland,Portugal,South Africa,Spain,Sweden,United Kingdom,United States
0,6mo_perf,-0.021563,0.103571,0.047724,0.22228,0.076412,0.155176,0.056426,-0.080645,0.099574,...,0.089209,-0.008658,0.092781,-0.04417,-0.04417,0.067239,0.050167,0.159822,0.041663,0.013761


In [12]:
yields_df =  yields_df.transpose()
yields_df = yields_df.reset_index(names = 'Country Name')
yields_df.columns = ['Country Name', '6 Month Performance']
yields_df = yields_df.drop(index = 0)
yields_df['6 Month Performance'] = yields_df['6 Month Performance'].astype(float).round(4)
yields_df.head()

,Country Name,6 Month Performance
1,Australia,-0.0216
2,Belgium,0.1036
3,Canada,0.0477
4,Denmark,0.2223
5,France,0.0764


In [13]:
final_df = pd.merge(merged_df, yields_df, on = 'Country Name', how = 'left')
final_df.head()

,Country Name,Credit Rating,E Score,S Score,G Score,ESG Score,6 Month Performance
0,Australia,100,0.547563,0.866146,0.821755,2.235464,-0.0216
1,Belgium,87,0.662349,0.773847,0.647842,2.084038,0.1036
2,Canada,100,0.484865,0.779746,0.752887,2.017498,0.0477
3,Denmark,100,0.750027,0.803905,0.846567,2.400499,0.2223
4,France,88,0.797240,0.764189,0.625450,2.186880,0.0764


In [14]:
final_df.to_csv('/Users/jamiewong/Documents/esg-financial-assistant/data/yields_cleaned.csv')